In [2]:
!pip install cvxpy
import pandas as pd
import numpy as np
import cvxpy as cp

  Using cached cvxpy-1.6.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.3 kB)
  Using cached osqp-0.6.7.post3-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.9 kB)
  Using cached clarabel-0.10.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.8 kB)
  Using cached scs-3.2.7.post2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached qdldl-0.1.7.post5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.7 kB)
Using cached cvxpy-1.6.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
Using cached clarabel-0.10.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
Using cached osqp-0.6.7.post3-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (300 kB)
Using cached scs-3.2.7.post2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.4 MB)
U

In [3]:
import cvxpy as cp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('data_project3(version_final).CSV')

# Extract relevant parameters from the dataset
energy_sources = data['Energy Source'].tolist()
production_capacity = data['Production (MW/h)'].tolist()
co2_emissions = data['CO? Emissions (unit/h)'].tolist()
air_pollution = data['Air Pollution (unit/h)'].tolist()
water_pollution = data['Water Pollution (unit/h)'].tolist()
production_cost = data['Cost to Produce ($/h)'].tolist()
construction_cost = data['Cost to build(million $)'].tolist()
maintenance_cost = data['Cost to Maintain ($/h)'].tolist()
transport_cost = data['Cost to Transport ($/M)'].tolist()

# Seasons and time slots
seasons = ['Winter', 'Spring', 'Summer', 'Fall']
time_slots = ['Morning', 'Noon', 'Evening', 'Night']

# Create a structured array for seasonal availability
availability = {}
for season in seasons:
    for time_slot in time_slots:
        column_name = f"{season} {time_slot}"
        availability[(season, time_slot)] = data[column_name].values / 6.0  # Normalize to 0-1

# Define dimensions
num_plant_types = len(energy_sources)  # K
num_seasons = len(seasons)  # I
num_time_slots = len(time_slots)  # J
max_plants_per_type = 10  # Maximum number of each plant type

# Create decision variables for plant allocation
# X[k,p] = 1 if we build plant p of type k, 0 otherwise
# We'll have max_plants_per_type potential slots for each plant type
X = cp.Variable((num_plant_types, max_plants_per_type), integer=True)

# Create variables for operation hours
# T[i,j,k,p] represents hours of operation for plant p of type k in season i, time slot j
T = cp.Variable((num_seasons, num_time_slots, num_plant_types, max_plants_per_type), nonneg=True)

# Set budget constraints
C_max = 20000  # Total construction budget in million $
P_max = 5000   # Maximum production budget per hour $

# Calculate demand levels (simplified version - can be adjusted)
# We'll use 70% of maximum possible production based on availability factors
demand = np.zeros((num_seasons, num_time_slots))
for i, season in enumerate(seasons):
    for j, time_slot in enumerate(time_slots):
        column_name = f"{season} {time_slot}"
        weighted_capacity = sum(production_capacity[k] * data[column_name].iloc[k]/6.0 for k in range(num_plant_types))
        demand[i, j] = 0.7 * weighted_capacity

# Objective function: Minimize environmental impact
# For each built plant, consider its environmental impact when operated
env_impact_coeffs = np.zeros((num_seasons, num_time_slots, num_plant_types, max_plants_per_type))
for i in range(num_seasons):
    for j in range(num_time_slots):
        for k in range(num_plant_types):
            for p in range(max_plants_per_type):
                env_impact_coeffs[i, j, k, p] = 0.3 * co2_emissions[k] + 0.3 * air_pollution[k] + 0.4 * water_pollution[k]

# Define the objective function
objective = cp.Minimize(cp.sum(cp.multiply(env_impact_coeffs, T)))

# Define constraints
constraints = []

# 1. Binary constraint for plant building decisions
constraints.append(X >= 0)
constraints.append(X <= 1)

# 2. Plant count limitation (optional - remove if not needed)
# Limit total number of plants that can be built
# constraints.append(cp.sum(X) <= total_max_plants)

# 3. Construction budget constraint
construction_costs = np.zeros((num_plant_types, max_plants_per_type))
for k in range(num_plant_types):
    construction_costs[k, :] = construction_cost[k]
constraints.append(cp.sum(cp.multiply(construction_costs, X)) <= C_max)

# 4. Demand constraints - total production must meet demand
for i in range(num_seasons):
    for j in range(num_time_slots):
        power_generated = 0
        for k in range(num_plant_types):
            for p in range(max_plants_per_type):
                power_generated += production_capacity[k] * T[i, j, k, p]
        constraints.append(power_generated >= demand[i, j])

# 5. Operation constraints - can only operate built plants
for i in range(num_seasons):
    for j in range(num_time_slots):
        for k in range(num_plant_types):
            for p in range(max_plants_per_type):
                constraints.append(T[i, j, k, p] <= 24 * X[k, p])  # 24 hours max per day

# 6. Availability constraints based on seasonal factors
for i, season in enumerate(seasons):
    for j, time_slot in enumerate(time_slots):
        for k in range(num_plant_types):
            column_name = f"{season} {time_slot}"
            avail_factor = data[column_name].iloc[k] / 6.0  # Normalize to 0-1
            for p in range(max_plants_per_type):
                constraints.append(T[i, j, k, p] <= 24 * avail_factor * X[k, p])

# 7. Production cost constraint
production_costs = np.zeros((num_seasons, num_time_slots, num_plant_types, max_plants_per_type))
for i in range(num_seasons):
    for j in range(num_time_slots):
        for k in range(num_plant_types):
            production_costs[i, j, k, :] = production_cost[k]

constraints.append(cp.sum(cp.multiply(production_costs, T)) <= P_max)

# Create and solve the problem
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GLPK_MI, verbose=True)

# Print results
print("\nStatus:", problem.status)
print("Optimal value:", problem.value)

# Count number of plants built per type
plants_built = np.zeros(num_plant_types, dtype=int)
for k in range(num_plant_types):
    plants_built[k] = int(sum(X.value[k, :]))

print("\nNumber of power plants to build:")
for k in range(num_plant_types):
    print(f"{energy_sources[k]}: {plants_built[k]}")

# Calculate total construction cost
total_construction_cost = 0
for k in range(num_plant_types):
    total_construction_cost += construction_cost[k] * sum(X.value[k, :])
print(f"\nTotal construction cost: ${total_construction_cost:.2f} million")

# Calculate total production for each plant type and season/time
total_production = np.zeros((num_seasons, num_time_slots, num_plant_types))
for i in range(num_seasons):
    for j in range(num_time_slots):
        for k in range(num_plant_types):
            for p in range(max_plants_per_type):
                total_production[i, j, k] += production_capacity[k] * T.value[i, j, k, p]

# Display total production by season and time slot
print("\nTotal Production (MW) by Season and Time Slot:")
for i, season in enumerate(seasons):
    for j, time_slot in enumerate(time_slots):
        total = sum(total_production[i, j, :])
        required = demand[i, j]
        print(f"{season} {time_slot}: {total:.2f} MW (Demand: {required:.2f} MW)")

# Visualize results
def create_plant_mix_chart():
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(energy_sources, plants_built)
    
    ax.set_ylabel('Number of Plants')
    ax.set_title('Optimal Power Plant Mix')
    ax.set_ylim(0, max(plants_built) * 1.2)
    
    # Add values on top of bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{int(height)}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig('plant_mix.png')
    plt.close()

def create_production_chart():
    # Calculate total production by energy source
    total_by_source = np.zeros(num_plant_types)
    for k in range(num_plant_types):
        for i in range(num_seasons):
            for j in range(num_time_slots):
                total_by_source[k] += total_production[i, j, k]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(energy_sources, total_by_source)
    
    ax.set_ylabel('Total Production (MW)')
    ax.set_title('Total Energy Production by Source')
    
    # Add values on top of bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{height:.1f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig('production_by_source.png')
    plt.close()

def create_season_production_chart():
    # Calculate total production by season
    season_production = np.zeros((num_seasons, num_plant_types))
    for i in range(num_seasons):
        for k in range(num_plant_types):
            for j in range(num_time_slots):
                season_production[i, k] += total_production[i, j, k]
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    bar_width = 0.15
    index = np.arange(len(seasons))
    
    for k in range(num_plant_types):
        offset = (k - num_plant_types/2 + 0.5) * bar_width
        ax.bar(index + offset, season_production[:, k], bar_width, label=energy_sources[k])
    
    ax.set_xlabel('Season')
    ax.set_ylabel('Total Production (MW)')
    ax.set_title('Energy Production by Season and Source')
    ax.set_xticks(index)
    ax.set_xticklabels(seasons)
    ax.legend()
    
    plt.tight_layout()
    plt.savefig('seasonal_production.png')
    plt.close()

# Create visualizations
try:
    create_plant_mix_chart()
    create_production_chart()
    create_season_production_chart()
    print("\nCharts saved: plant_mix.png, production_by_source.png, seasonal_production.png")
except Exception as e:
    print(f"Error creating charts: {e}")

# Calculate environmental impact
total_co2 = 0
total_air = 0
total_water = 0
for i in range(num_seasons):
    for j in range(num_time_slots):
        for k in range(num_plant_types):
            for p in range(max_plants_per_type):
                if X.value[k, p] > 0.5:  # If plant is built
                    total_co2 += co2_emissions[k] * T.value[i, j, k, p]
                    total_air += air_pollution[k] * T.value[i, j, k, p]
                    total_water += water_pollution[k] * T.value[i, j, k, p]

print("\nEnvironmental Impact:")
print(f"Total CO2 Emissions: {total_co2:.2f} units")
print(f"Total Air Pollution: {total_air:.2f} units")
print(f"Total Water Pollution: {total_water:.2f} units")

# Calculate total costs
total_production_cost = 0
total_maintenance = 0
total_transport = 0
for i in range(num_seasons):
    for j in range(num_time_slots):
        for k in range(num_plant_types):
            for p in range(max_plants_per_type):
                if X.value[k, p] > 0.5:  # If plant is built
                    total_production_cost += production_cost[k] * T.value[i, j, k, p]
                    total_maintenance += maintenance_cost[k] * T.value[i, j, k, p]
                    total_transport += transport_cost[k] * T.value[i, j, k, p]

print("\nCost Analysis:")
print(f"Total Production Cost: ${total_production_cost:.2f}")
print(f"Total Maintenance Cost: ${total_maintenance:.2f}")
print(f"Total Transport Cost: ${total_transport:.2f}")

                                     CVXPY                                     
                                     v1.6.3                                    
(CVXPY) Mar 14 06:00:18 PM: Your problem has 1190 variables, 2398 constraints, and 0 parameters.
(CVXPY) Mar 14 06:00:19 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 14 06:00:19 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 14 06:00:19 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Mar 14 06:00:19 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 14 06:00:19 PM: Compiling problem (target solver=GLPK_M

/opt/conda/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:418: UserWarning: The problem has an expression with dimension greater than 2. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


(CVXPY) Mar 14 06:00:19 PM: Applying reduction CvxAttr2Constr
(CVXPY) Mar 14 06:00:20 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Mar 14 06:00:24 PM: Applying reduction GLPK_MI
(CVXPY) Mar 14 06:00:24 PM: Finished problem compilation (took 5.570e+00 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Mar 14 06:00:24 PM: Invoking solver GLPK_MI  to obtain a solution.
      0: obj =   0.000000000e+00 inf =   1.284e+01 (16)
     95: obj =   2.470201813e+02 inf =   0.000e+00 (0)
*   309: obj =   2.903750671e+01 inf =   8.509e-15 (0)
+   309: mip =     not found yet >=              -inf        (1; 0)
+   400: >>>>>   2.903750671e+01 >=   2.903750671e+01   0.0% (20; 0)
+   400: mip =   2.903750671e+01 >=     tree is empty   0.0% (0; 39)
-----------------------------------------------

In [ ]:
C = 